# Use Case - Collection Search (IDN)

This chapter provides a comprehensive and detailed process about how to implement a WGISS
OpenSearch client, which includes how to retrieve the OSDD for the collection of interest, and
how to build an OpenSearch request. 

In [54]:
import re
import json, requests, xml
# import pandas as pd
# import ipywidgets as widgets

from xml.dom import minidom
# from IPython.display import Image
from xml.etree import ElementTree
# from IPython.display import HTML
from IPython.display import Markdown as md

In [55]:
def get_api_request(template, os_querystring):
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and return as short HTTP URL without empty parameters.
  
  # print("URL template: " + template)
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  # print("API request: " + template)
            
  return (template)

## IDN Systems


The IDN, CMR OpenSearch (for IDN), and the GCMD’s Keyword Management Service (KMS)
only have operational systems which end-users can access.

- IDN site is available to all users.  Location: https://idn.ceos.org/
- OpenSearch API for IDN (via CMR). Production instance is available to all users.  Location: https://cmr.earthdata.nasa.gov/opensearch/
- KMS - production instance is available to all users.  Location: https://gcmd.earthdata.nasa.gov/kms/capabilities?format=html

The IDN site search interface and the CMR OpenSearch production instances will provide access to all collections which have been registered in the IDN. The KMS production instance will provide access to all approved GCMD keywords registered by IDN providers.

## Retrieve Collections via IDN OpenSearch

CEOS OpenSearch supports searching for collections through the IDN. Searching for granules in a specific collection is supported at the data partners via the Granule Gateways (see chapter
“CWIC” and chapter “FedEO”). It executes a collection or inventory search, as appropriate, and returns the matching results.  In order to create a valid request, clients have to obtain the IDN OpenSearch OSDD and fill request
parameters with proper values.

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [56]:
URL_OSDD = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

The template of the OpenSearch request is available under the `<Url>` element corresponding to the media type (Atom) in the OSDD and is included below.

In [57]:
response = requests.get( URL_OSDD )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><os:OpenSearchDescription xmlns:atom="http://www.w3.org/2005/Atom" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">  
    <os:ShortName>CMR Collections</os:ShortName>  
    <os:Description>NASA CMR Collection search using geo, time and parameter extensions</os:Description>  
    <os:Contact>echodev@echo.nasa.gov</os:Contact>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="application/atom+xml">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.html?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="text/html">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Query role="example" searchTerms="Amazon River Basin Precipitation, 1972-1992" title="Sample search"/>  
    <os:Attribution>NASA CMR</os:Attribution>  
    <os:SyndicationRight>open</os:SyndicationRight>  
    <os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC collection pageOffset=1 indexOffset=0</os:Tags>  
</os:OpenSearchDescription>
```


**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters. 

An example request can be formed as follows.

In [58]:
# find URL template for collection search
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

In [59]:
request_url = get_api_request(collection_template, {'count': '10', 'searchTerms': 'Landsat_8'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat_8&numberOfResults=10&clientId=ceosOpenSearchDoc'

In [60]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-06T09:53:12.778Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; Landsat_8</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat_8&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat_8&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat_8&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="Landsat_8" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>1</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1235542031-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat 8</title>      
          <summary type="text">The Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) are onboard the Landsat 8 satellite, have acquired images of the Earth since  February 2013. The sensors collect images of the Earth with a 16-day repeat cycle, referenced to the Worldwide Reference System-2. The approximate scene size is 170 km north-south by 183 km east-west (106 mi by 114 mi).

Landsat 8 image data files consist of 11 spectral bands with a spatial resolution of 30 meters for bands 1-7 and bands 9-11; 15-meters for the panchromatic band 8. Delivered Landsat 8 Level-1 data typically include both OLI and TIRS data files; however, there may be OLI-only and/or TIRS-only scenes in the USGS archive.  A Quality Assurance (QA.tif) band is also included. This file provides bit information regarding conditions that may affect the accuracy and usability of a given pixel – clouds, water or snow, for example.</summary>      
          <updated/>      
          <link href="http://glovis.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://landsat.usgs.gov/landsat8.php" hreflang="en-US" rel="describedBy"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1235542031-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1235542031-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-11T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8</echo:datasetId>      
          <echo:shortName>Landsat_8</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>Landsat_8</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:processingLevelId>Not Provided</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-82.71 -180 82.74 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>0.97999995</relevance:score>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.provider</echo:tagKey>         
               <echo:data>&quot;USGSLSI&quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.prod</echo:tagKey>         
               <echo:data>&quot; &quot;</echo:data>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


**Step 3**  
>  From the IDN OpenSearch response obtain the OSDD endpoint for the collection by parsing the href attribute in the `<link rel="search" type="application/opensearchdescription+xml" />` element. Note that the OSDD endpoint may refer to CWIC or FedEO. 

In [61]:
request_url = get_api_request(collection_template, {'count': '10', 'geo:uid': 'C1235542031-USGS_LTA'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&numberOfResults=10&uid=C1235542031-USGS_LTA&clientId=ceosOpenSearchDoc'

In [62]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-06T09:53:13.788Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: uid =&gt; C1235542031-USGS_LTA</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&amp;numberOfResults=10&amp;uid=C1235542031-USGS_LTA&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&amp;numberOfResults=10&amp;uid=C1235542031-USGS_LTA&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&amp;numberOfResults=10&amp;uid=C1235542031-USGS_LTA&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query geo:uid="C1235542031-USGS_LTA" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>1</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1235542031-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat 8</title>      
          <summary type="text">The Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) are onboard the Landsat 8 satellite, have acquired images of the Earth since  February 2013. The sensors collect images of the Earth with a 16-day repeat cycle, referenced to the Worldwide Reference System-2. The approximate scene size is 170 km north-south by 183 km east-west (106 mi by 114 mi).

Landsat 8 image data files consist of 11 spectral bands with a spatial resolution of 30 meters for bands 1-7 and bands 9-11; 15-meters for the panchromatic band 8. Delivered Landsat 8 Level-1 data typically include both OLI and TIRS data files; however, there may be OLI-only and/or TIRS-only scenes in the USGS archive.  A Quality Assurance (QA.tif) band is also included. This file provides bit information regarding conditions that may affect the accuracy and usability of a given pixel – clouds, water or snow, for example.</summary>      
          <updated/>      
          <link href="http://glovis.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://landsat.usgs.gov/landsat8.php" hreflang="en-US" rel="describedBy"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1235542031-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1235542031-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-11T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8</echo:datasetId>      
          <echo:shortName>Landsat_8</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:entryId>Landsat_8</echo:entryId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:processingLevelId>Not Provided</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-82.71 -180 82.74 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.provider</echo:tagKey>         
               <echo:data>&quot;USGSLSI&quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.prod</echo:tagKey>         
               <echo:data>&quot; &quot;</echo:data>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


Obtain the OSDD endpoint for the granule search for this collection by parsing the href attribute in `<link rel="search"
type="application/opensearchdescription+xml" >`

In [63]:
root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search with Atom response
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml" />
    
```


Extract the URL of the OSDD endpoint from the `<link>`.

In [64]:
url_osdd = el.attrib['href']
el.attrib['href']

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA'

**Step 4**  
>  From the collection OSDD found in the IDN OpenSearch response, formulate a valid granule search request. How to do this is explained in sections (“CWIC”) and (“FedEO”).  

In principle, CEOS OpenSearch clients compliant with CEOS OpenSearch Best Practices should
not care whether the second step (i.e. Granule search) redirects to CWIC or FedEO as both
endpoints provide the same interface which clients should discover by obtaining the OSDD.
Search parameters in the URL template which are specific to one of both endpoints (i.e. not defined
in the CEOS Best Practice for OpenSearch or belonging to a foreign namespace) can be left empty
when preparing the granule search request.

## Available Collection Search Criteria

CEOS OpenSearch is used as the IDN’s collections search implementation based on
the [OpenSearch 1.1 (Draft 5) specification](https://github.com/dewitt/opensearch/blob/master/opensearch-1-1-draft-6.md) and is compliant with the CEOS OpenSearch Best
Practices. The IDN OpenSearch API allows clients to formulate OpenSearch compliant queries
against the IDN collections and specify the desired search results format as OpenSearch
compliant Atom or HTML. The IDN OpenSearch API implements the following search fields for
users’ queries:

**Table of collection search criteria**

In [65]:
%%html

<table>

<tr><th>Request<br/>
Parameter</th><th align="left">
Description
</th><th>Value &
Cardinality<br/>
(M) = mandatory<br/>
(O) = optional
</th><th>
Type</th></tr>


<tr><td>boundingBox</td><td align="left"> Inventory with a spatial extent overlapping
this bounding box
</td><td>(O)</td><td> geo:box</td></tr>

<tr><td>keyword</td><td align="left"> Inventory with terms expressed by these
search terms
</td><td>(O)</td><td> os:searchTerms</td></tr>

<tr><td>instrument</td><td align="left"> Inventory associated with a satellite
instrument expressed by this short name
</td><td>(O)</td><td> echo:instrument</td></tr>


<tr><td>satellite</td><td align="left"> Inventory associated with a
Satellite/platform expressed by this short
name
</td><td>(O)</td><td> eo:platform</td></tr>

<tr><td>geometry</td><td align="left"> Inventory with a spatial extent overlapping
this geometry
</td><td>(O)</td><td> geo:geometry</td></tr>

<tr><td>placeName</td><td align="left"> Inventory with a spatial location described
by this name
</td><td>(O)</td><td> geo:name</td></tr>

<tr><td>startTime</td><td align="left"> Inventory with a temporal extent containing
this start time
</td><td>(O)</td><td> time:start</td></tr>

<tr><td>endTime</td><td align="left"> Inventory with a temporal extent containing
this end time
</td><td>(O)</td><td> time:end</td></tr>


<tr><td>cursor</td><td align="left"> Start page for the search result </td><td>(O)</td><td> os:startPage</td></tr>

<tr><td>numberOfResults</td><td align="left"> Maximum number of records in the search
result
</td><td>(O)</td><td> os:count</td></tr>


<tr><td>offset</td><td align="left"> 0 - based offset used to skip the specified
number of results in the search result set
</td><td>(O)</td><td> os:startIndex</td></tr>

<tr><td>uid</td><td align="left"> Inventory associated with this unique ID</td><td>(O)</td><td> geo:uid</td></tr>
<tr><td>hasGranules</td><td align="left"> Inventory with granules</td><td>(O)</td><td> echo:hasGranules</td></tr>
<tr><td>isCwic</td><td align="left"> Inventory related to CWIC</td><td>(O)</td><td> echo:isCwic</td></tr>
<tr><td>isGeoss</td><td align="left"> Inventory related to GEOSS</td><td>(O)</td><td> echo:isGeoss</td></tr>
<tr><td>isCeos</td><td align="left"> Inventory related to CEOS</td><td>(O)</td><td></td></tr>
<tr><td>isEosdis</td><td align="left"> Inventory related to EOSDIS</td><td>(O)</td><td> echo:isEosdis</td></tr>
<tr><td>provider</td><td align="left"> Inventory associated with a provider</td><td>(O)</td><td> echo:provider</td></tr>
<tr><td>clientId</td><td align="left"> Client identifier to be used for metrics</td><td>(O)</td><td> cswOpenSearchDoc</td></tr>

</table>

Request Parameter,Description,Value & Cardinality (M) = mandatory (O) = optional,Type
boundingBox,Inventory with a spatial extent overlapping this bounding box,(O),geo:box
keyword,Inventory with terms expressed by these search terms,(O),os:searchTerms
instrument,Inventory associated with a satellite instrument expressed by this short name,(O),echo:instrument
satellite,Inventory associated with a Satellite/platform expressed by this short name,(O),eo:platform
geometry,Inventory with a spatial extent overlapping this geometry,(O),geo:geometry
placeName,Inventory with a spatial location described by this name,(O),geo:name
startTime,Inventory with a temporal extent containing this start time,(O),time:start
endTime,Inventory with a temporal extent containing this end time,(O),time:end
cursor,Start page for the search result,(O),os:startPage
numberOfResults,Maximum number of records in the search result,(O),os:count


Also, client developers are able to query with specific tags: isCeos, isCwic, isGeoss, and isFedEO.
Tagging allows arbitrary sets of collections to be grouped under a single namespace value. The
sets of collections can be recalled later when searching by tag fields.

IDN query examples:

- GET the first 10 IDN collections with results in the Atom format:
    https://cmr.earthdata.nasa.gov/opensearch/collections.atom?numberOfResults=10&clientId=cswOpenSearchDoc
- GET the first 10 IDN collections containing the GCMD instrument keyword MODIS
    with results in the Atom output format:
    https://cmr.earthdata.nasa.gov/opensearch/collections.atom?instrument=MODIS&numberOfResults=10&clientId=cswOpenSearchDoc
- GET the first 10 CWIC IDN collections containing the GCMD instrument keyword
    MODIS with results in the HTML format:
    https://cmr.earthdata.nasa.gov/opensearch/collections?instrument=MODIS&isCwic=true&numberOfResults=10&clientId=cswOpenSearchDoc

